In [1]:
#libraries:

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import urllib.request
import pandas as pd

In [6]:
#Initialising Lists
all_links = []
all_pages = []

In [7]:
%%time
#Scraping Pages
link = 'https://pk.sapphireonline.pk/collections/menswear'
all_pages.append(link)
base_link = 'https://pk.sapphireonline.pk'
while (True):
        uClient = uReq(link)
        all_web = uClient.read()
        uClient.close()
        page_soup = soup(all_web, "html.parser")
        pages = page_soup.find('div', {'class':'infinite-scrolling'})
        if (pages == None):
            break
        link = base_link + pages.a['href']
        all_pages.append(link)

all_pages
        

Wall time: 4.81 s


['https://pk.sapphireonline.pk/collections/menswear',
 'https://pk.sapphireonline.pk/collections/menswear?page=2',
 'https://pk.sapphireonline.pk/collections/menswear?page=3',
 'https://pk.sapphireonline.pk/collections/menswear?page=4']

In [8]:
%%time
#Scraping All Product Links
product_container = []
for link in all_pages:
    uClient = uReq(link)
    all_web = uClient.read()
    uClient.close()
    page_soup = soup(all_web, "html.parser")
    for product in page_soup.findAll("div", {"class":"grid-item col-6 col-md-4 col-xl-3 four-columns"}):
        product_container.append(product)
    #Retrieving All Product Links
    for link in range(len(product_container)):
        x = base_link + product_container[link].a['href']
        all_links.append(x)

all_rows = []
len(all_links)

Wall time: 5.65 s


442

In [46]:
%%time
#Scraping All Products


h = 0
x = 0
faultylinks = []
for link in all_links:
    h += 1
    if (h%5 == 0):    
        print('working...' + str(x))
    
    try:
        uClient = uReq(link)
        all_web = uClient.read()
        uClient.close()
        name = None
        des1 = None
        des2 = None
        detail = None
        color = None
        fabric = None
        oldprice = None
        newprice = None
        all_sizes = None
        extra_detail = None
        product_soup = soup(all_web, "html.parser")
        name = product_soup.find("h2", {"itemprop":"name"}).text.strip()
        content = product_soup.find("section", {"id":'content1'})('p')
        name = product_soup.find("h2", {"itemprop":"name"}).text.strip()
        if (len(content) != 1 and len(content) <= 5):
            if (content[1].text != 'Ready To Wear'):
                if (content[0].strong != None):
                    des1 = content[0].strong.text
                else:
                    b = content[0].text.find("\n")
                    des1 = content[0].text[:b].strip()

                des2 = content[0].text.replace(des1, "").strip()
                details = content[1].text
                z = details.find("Color")
                detail = details[8:z].strip()
                l = details.find("Fabric")
                color = details[z+6:l].strip()
                fabric = details[l+7:].strip()
                if len(content) == 3:
                    extra_detail = content[2].text
            else:
                des1 = content[2].text.strip()
                des2 = content[0].text.strip()
                detail = content[1].text.strip()
                z = content[3].text.find("Fabric")
                color = content[3].text[:z].replace("Color:", "").strip()
                fabric = content[3].text[z:].replace("Fabric:", "").strip()
        elif (len(content) >= 6):
            des1 = content[3].text.strip()
            des2 = content[1].text.strip()
            detail = content[4].text.strip()
            z = content[5].text.find("Fabric")
            color = content[5].text[:z].replace("Color:", "").strip()
            fabric = content[5].text[z:].replace("Fabric:", "").strip()

        else:
            content = content[0]
            des1 = content.strong.text.strip()
            content = content.text
            z = content.find("Details:")
            l = content.find("Color")
            k = content.find("Fabric")
            j = content.find("SIZE")
            des2 = content[len(des1):z].strip()
            detail = content[z:l].replace("Details:", "").strip()
            color = content[l:k].replace("Color:", "").strip()
            fabric = content[k:j].replace("Fabric:", "").strip()


        prices = product_soup.find("div",{"class":"prices"})('span')
        oldprice = prices[1].text.strip()
        newprice = prices[2].text.strip()
        sizes = product_soup.find_all("div", {'class':'swatch-element'})
        all_sizes = []
        types = []
        for size in sizes:
            c = size.label.text.strip()
            if (c == 'XXS' or c == 'XS' or c == 'S' or c == 'M' or c == 'L' or c == 'XL' or c == 'XLL'):
                all_sizes.append(c)
            else:
                types.append(c)

        all_sizes = ','.join(all_sizes)
        types = ','.join(types)
        Images
        images = product_soup.find_all("a", {"class":"fancybox"})
        y = 0
        for a in images:
            file_name = str(x) + "." + str(y) + ".jpg"
            #print(a['href'] + file_name)
            y += 1
            urllib.request.urlretrieve("https:" + a['href'], file_name)
        
        x += 1
        row = {'Name':name,'Detail':detail,'Main-Description': des1,'Sub-Description':des2, 'Old-Price': oldprice, 'New-Price': newprice,'Color': color, 'Fabric' : fabric, 'Types': types, 'All Sizes' : all_sizes,"Extra Detail" : extra_detail}
        all_rows.append(row)
    except:
        print(link)
        faultylinks.append(link)
        pass

working...4
working...9
working...14
working...19
working...24
working...29
working...34
working...39
working...44
working...49
working...54
working...59
working...64
working...69
working...74
working...79
working...84
working...89
working...94
working...99
working...104
working...109
working...114
working...119
working...124
working...129
working...134
working...139
working...144
working...149
working...154
working...159
working...164
working...169
working...174
working...179
working...184
working...189
working...194
working...199
working...204
working...209
working...214
working...219
working...224
working...229
working...234
working...239
working...244
working...249
working...254
working...259
working...264
working...269
working...274
working...279
working...284
working...289
working...294
working...299
working...304
working...309
working...314
working...319
working...324
working...329
working...334
working...339
working...344
working...349
working...354
working...359
working...364


In [45]:
%%time
#Scraping 1 product
#print(all_links[1])
link = 'https://pk.sapphireonline.pk/collections/menswear/products/sm0246-long-boot'
uClient = uReq(link)
all_web = uClient.read()
uClient.close()
name = None
des1 = None
des2 = None
detail = None
color = None
fabric = None
oldprice = None
newprice = None
all_sizes = None
extra_detail = None
extra_detail2 = None
product_soup = soup(all_web, "html.parser")
name = product_soup.find("h2", {"itemprop":"name"}).text.strip()
content = product_soup.find("section", {"id":'content1'})('p')
name = product_soup.find("h2", {"itemprop":"name"}).text.strip()
if (len(content) != 1 and len(content) <= 5):
    if (content[1].text != 'Ready To Wear'):
        if (content[0].strong != None):
            des1 = content[0].strong.text
        else:
            b = content[0].text.find("\n")
            des1 = content[0].text[:b].strip()

        des2 = content[0].text.replace(des1, "").strip()
        details = content[1].text
        z = details.find("Color")
        detail = details[8:z].strip()
        l = details.find("Fabric")
        color = details[z+6:l].strip()
        fabric = details[l+7:].strip()
        if len(content) == 3:
            extra_detail = content[2].text
    else:
        des1 = content[2].text.strip()
        des2 = content[0].text.strip()
        detail = content[1].text.strip()
        z = content[3].text.find("Fabric")
        color = content[3].text[:z].replace("Color:", "").strip()
        fabric = content[3].text[z:].replace("Fabric:", "").strip()
elif (len(content) >= 6):
    des1 = content[3].text.strip()
    des2 = content[1].text.strip()
    detail = content[4].text.strip()
    z = content[5].text.find("Fabric")
    color = content[5].text[:z].replace("Color:", "").strip()
    fabric = content[5].text[z:].replace("Material:", "").strip()
    
else:
    content = content[0]
    des1 = content.strong.text.strip()
    content = content.text
    z = content.find("Details:")
    l = content.find("Color")
    k = content.find("Fabric")
    j = content.find("SIZE")
    des2 = content[len(des1):z].strip()
    detail = content[z:l].replace("Details:", "").strip()
    color = content[l:k].replace("Color:", "").strip()
    fabric = content[k:j].replace("Fabric:", "").strip()
    
    
prices = product_soup.find("div",{"class":"prices"})('span')
oldprice = prices[1].text.strip()
newprice = prices[2].text.strip()
sizes = product_soup.find_all("div", {'class':'swatch-element'})
all_sizes = []
types = []
for size in sizes:
    c = size.label.text.strip()
    if (c == 'XXS' or c == 'XS' or c == 'S' or c == 'M' or c == 'L' or c == 'XL' or c == 'XLL'):
        all_sizes.append(c)
    else:
        types.append(c)
    
all_sizes = ','.join(all_sizes)
#if (types != []):
types = ','.join(types)
    
row = {'Name':name,'Detail':detail,'Main-Description': des1,'Sub-Description':des2, 'Old-Price': oldprice, 'New-Price': newprice,'Color': color, 'Fabric' : fabric, 'Types':types, 'All Sizes' : all_sizes,"Extra Detail-1" : extra_detail, "Extra Detail-2": extra_detail2}

print(name, des1, des2,detail,color,fabric, oldprice, newprice, all_sizes)


Wall time: 1.68 s


In [47]:
products = pd.DataFrame(all_rows, columns = ['Name','Detail','Main-Description', 'Sub-Description','Old-Price','New-Price','Color', 'Fabric','Types','All Sizes',"Extra Detail"])

In [48]:
products

,Name,Detail,Main-Description,Sub-Description,Old-Price,New-Price,Color,Fabric,Types,All Sizes,Extra Detail
0,Gingham,,Chelsea Boot,Leather chelsea boots in classic black are an ...,"Rs.4,490.00","Rs.2,918.50",Black,l: Leather,NaN,"40,41,42,43,44,45",NaN
1,Aqua,"Plain Front, Plain Back, Band Neck",Waist Coat,This collarless blue waistcoat comes in luxe s...,"Rs.4,490.00","Rs.2,918.50",Blue,Silk Karandi,Regular Fit,"S,M,L,XL",None
2,Byzantium,"Plain Front with Embroidery on Arm Hole, Plain...",Waist Coat,"An aubergine waistcoat in tropic fabric, with ...","Rs.4,490.00","Rs.2,918.50",Deep Purple,Tropical fabric,Regular Fit,"S,M,L,XL",None
3,Cosmos,"Plain Front, Plain Back, Band Neck",Waist Coat,A classic raven-black tropical fabric piece wi...,"Rs.4,490.00","Rs.2,918.50",Black,Tropical,Regular Fit,"S,M,L,XL",None
4,Sahara,"Plain Front, Plain Back, Full Sleeves, Band Ne...",2-Piece Kurta Shalwar,A classy two piece suit in sandy beige feature...,"Rs.5,990.00","Rs.3,893.50",Beige,Toyobo,Regular Fit,"XS,S,M,L,XL",None
5,Depths,"Plain Front, Plain Back, Full Sleeves, Collare...",2-Piece Kurta Shalwar,This spun polyester collared kurta set comes i...,"Rs.5,990.00","Rs.3,893.50",Navy Blue,Toyobo,Regular Fit,"XS,S,M,L,XL",None
6,Everest,"Plain Front, Plain Back, Full Sleeves, Collare...",2-Piece Kurta Shalwar,A pure white kurta suit with a classic collar ...,"Rs.5,990.00","Rs.3,893.50",White,Toyobo,Regular Fit,"XS,S,M,L,XL",None
7,Enrich,"Plain Front, Plain Back, Full Sleeves, Band Ne...",2-Piece Kurta Shalwar,A grey two piece set featuring a kurta with a ...,"Rs.5,990.00","Rs.3,893.50",Grey,Toyobo,Regular Fit,"XS,S,M,L,XL",None
8,Ingenue,"Plain Front, Plain Back, Full Sleeves, Band Ne...",2-Piece Kurta Shalwar,Make a statement with this mauve band-collared...,"Rs.5,990.00","Rs.3,893.50",Mauve,Toyobo,Regular Fit,"XS,S,M,L,XL",None
9,Tourmaline,"Plain Front with Embroidered Neckline & Cuffs,...",Straight Kurta,An ebony jacquard kurta with embroidery worked...,"Rs.2,490.00","Rs.1,618.50",Black,100% Cotton Jacquard,"Regular Fit,Slim Fit","XS,S,M,L,XL",None


In [ ]:
products.to_csv('Sapphire-MensWear.csv', encoding='utf-8')